In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

file_path = "NALUS_03-03-2025.csv"

df = pd.read_csv(
    "NALUS_03-03-2025.csv",
    encoding='cp1250',      # Using Latin-1 encoding as specified
    delimiter=';',          # Semicolon delimiter
    quotechar='"',          # Double quotes for quoted fields
    escapechar='\\',        # Backslash as escape character
    on_bad_lines='warn',    # Warn about problematic lines but don't fail
    low_memory=False,       # Load entire file into memory for better parsing
    index_col=False
)

df

,Sp.zn.,Paralelní citace (Sb.),Paralelní citace (SbNU),ECLI,Soudce zpravodaj,Navrhovatel,Populární název,Datum rozhodnutí,Datum vyhlášení,Datum podání,Datum zpřístupnění,Vztah k předpisům,Forma rozhodnutí,Význam,Výrok,Předmět řízení,Věcný rejstřík
0,II.ÚS 2694/24,NaN,NaN,ECLI:CZ:US:2025:2.US.2694.24.2,Ronovská Kateřina,STĚŽOVATEL - PO,Náležité posouzení právního poměru mezi soukro...,19. 2. 2025,NaN,27. 9. 2024,28. 2. 2025,"111/1998 Sb., § 59, § 36, § 41\n2/1993 Sb./Sb....",Nález,3,vyhověno\nodmítnuto pro zjevnou neopodstatněno...,právo na soudní a jinou právní ochranu /soudní...,NaN
1,IV.ÚS 2048/23,NaN,NaN,ECLI:CZ:US:2025:4.US.2048.23.1,Baxa Josef,STĚŽOVATEL - PO,Naplnění předpokladů přípustnosti dovolání,12. 2. 2025,NaN,31. 7. 2023,25. 2. 2025,"2/1993 Sb./Sb.m.s., čl. 36 odst.1\n266/1994 Sb...",Nález,3,vyhověno\nodmítnuto pro nepřípustnost,právo na soudní a jinou právní ochranu /právo ...,NaN
2,IV.ÚS 2672/23,NaN,NaN,ECLI:CZ:US:2025:4.US.2672.23.3,Křesťanová Veronika,STĚŽOVATEL - PO,Ochrana pověsti právnických osob,6. 2. 2025,NaN,9. 10. 2023,27. 2. 2025,"1/1993 Sb./Sb.m.s., čl. 89 odst.2\n2/1993 Sb./...",Nález,3,vyhověno,základní práva a svobody/ochrana lidské důstoj...,NaN
3,IV.ÚS 2093/24,NaN,NaN,ECLI:CZ:US:2025:4.US.2093.24.1,Kühn Zdeněk,STĚŽOVATEL - FO,Smlouva o vzdělávání na soukromé vysoké škole ...,5. 2. 2025,NaN,24. 7. 2024,25. 2. 2025,"111/1998 Sb., § 58, § 59, § 40\n2/1993 Sb./Sb....",Nález,3,vyhověno\nodmítnuto pro nepřípustnost,právo na soudní a jinou právní ochranu /právo ...,NaN
4,I.ÚS 2364/24,NaN,NaN,ECLI:CZ:US:2025:1.US.2364.24.2,Langášek Tomáš,STĚŽOVATEL - FO,Rozhodování o (a)symetrickém rozložení střídav...,5. 2. 2025,NaN,22. 8. 2024,26. 2. 2025,"2/1993 Sb./Sb.m.s., čl. 32 odst.4, čl. 36 odst...",Nález,3,vyhověno,"hospodářská, sociální a kulturní práva/právo n...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,III.ÚS 743/19,NaN,N 118/94 SbNU 400,ECLI:CZ:US:2019:3.US.743.19.1,Rychetský Pavel,STĚŽOVATEL - FO,Povinnost soudu doručit odvolání proti rozhodn...,25. 6. 2019,NaN,1. 3. 2019,12. 7. 2019,"1/1993 Sb./Sb.m.s., čl. 96 odst.1\n2/1993 Sb./...",Nález,3,vyhověno,právo na soudní a jinou právní ochranu /sprave...,NaN
223,Pl.ÚS 25/15,186/2019 Sb.,N 114/94 SbNU 360,ECLI:CZ:US:2019:Pl.US.25.15.1,Uhlíř David,SOUD - KS Praha,Ústavnost souhlasu vlastníka místní komunikace...,18. 6. 2019,NaN,25. 9. 2015,8. 7. 2019,"1/1993 Sb./Sb.m.s., čl. 101 odst.4\n13/1997 Sb...",Nález,2,zamítnuto,právo na soudní a jinou právní ochranu /právo ...,NaN
224,IV.ÚS 2503/18,NaN,N 112/94 SbNU 341,ECLI:CZ:US:2019:4.US.2503.18.1,Šimáčková Kateřina,STĚŽOVATEL - FO,K povinnosti obecných soudů reflektovat nosné ...,12. 6. 2019,NaN,23. 7. 2018,3. 7. 2019,"1/1993 Sb./Sb.m.s., čl. 89 odst.2\n2/1993 Sb./...",Nález,3,vyhověno,procesní otázky řízení před Ústavním soudem/zá...,NaN
225,I.ÚS 3926/16,NaN,N 108/94 SbNU 311,ECLI:CZ:US:2019:1.US.3926.16.1,Uhlíř David,STĚŽOVATEL - FO,Formulace naléhavého právního zájmu v určovací...,11. 6. 2019,NaN,28. 11. 2016,11. 7. 2019,"2/1993 Sb./Sb.m.s., čl. 11 odst.1, čl. 36 odst...",Nález,3,vyhověno\nodmítnuto pro nepřípustnost,právo na soudní a jinou právní ochranu /soudní...,NaN


In [ ]:
def ecli_to_url(ecli):
    match = re.search(r'(\d+)\.US\.(\d+)\.(\d+)\.(\d+)', ecli)
    if match:
        registry, case, year, suffix = match.groups()
        return f"https://nalus.usoud.cz:443/Search/GetText.aspx?sz={registry}-{case}-{year}_{suffix}"
    return None

# Test
ecli = "ECLI:CZ:US:2025:2.US.2694.24.2"
print(ecli_to_url(ecli))

https://nalus.usoud.cz:443/Search/GetText.aspx?sz=2-2694-24_2


In [67]:
# URL from the comment, which is also retrievable via ecli_to_url function
url = "https://nalus.usoud.cz:443/Search/GetText.aspx?sz=2-2694-24_2"


def download_text(url):
    response = requests.get(url)
    response.encoding = 'utf-8'  # Set encoding

    # Parse HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract and print the text content
    content = soup.get_text()
    content = re.sub(r'\n{3,}', '\n\n\n', content)
    content = re.sub(r' {4,}', '   ', content)
    
    return content

download_text(ecli_to_url(ecli))

'\n\n\r\n\tNALUS - databáze rozhodnutí Ústavního soudu\r\n\n\n\r\n   \xa0\xa0\r\n   \r\n   \xa0\xa0\r\n   \n\n\nII.ÚS 2694/24 ze dne 19. 2. 2025\n\n\nNáležité posouzení právního poměru mezi soukromou vysokou školou a studentkou ze strany Nejvyššího soudu\n\n\n\xa0\n\n\nČeská republika\n\n\nNÁLEZ\n\n\nÚstavního soudu\n\n\nJménem republiky\n\n\n\xa0\n\n\n\r\nÚstavní soud rozhodl v senátu složeném z předsedy Pavla Šámala a soudkyň Kateřiny Ronovské (soudkyně zpravodajky) a Dity Řepkové o ústavní stížnosti stěžovatelky Vysoké školy ekonomie a managementu, a.s., sídlem Nárožní 2600/9a, Praha 5, zastoupené Mgr. Martinem Zástěrou, advokátem se sídlem Radhošťská 1942/2, Praha 3, proti rozsudku Nejvyššího soudu č. j. 33 Cdo 2443/2023-328 ze dne 30. 7. 2024, spojené s návrhem na zrušení ustanovení § 59 zákona č. 111/1998 Sb., o vysokých školách a o změně a doplnění dalších zákonů (zákon o vysokých školách), ve znění pozdějších předpisů, za účasti Nejvyššího soudu, jako účastníka řízení, a Karolí

In [15]:
import pandas as pd

df = pd.read_json("NALUS_03-03-2025.json")

# show lines where pl is included in the Sp.zn. column
df[df["Sp.zn."].str.contains("Pl")].iloc[0].text

'\n\n\r\n\tNALUS - databáze rozhodnutí Ústavního soudu\r\n\n\n\r\n   \xa0\xa0\r\n   \r\n   \xa0\xa0\r\n   \n\n\nPl.ÚS 26/24 ze dne 15. 1. 2025\n\n\nOchrana pověsti právnických osob\n\n\n\xa0\n\n\nČeská republika\n\n\nNÁLEZ\n\n\nÚstavního soudu\n\n\nJménem republiky\n\n\n\xa0\n\n\n\r\nÚstavní soud rozhodl v plénu složeném z předsedy soudu Josefa Baxy a soudkyň a soudců Lucie Dolanské Bányaiové, Josefa Fialy, Milana Hulmáka, Jaromíra Jirsy (soudce zpravodaje), Zdeňka Kühna, Veroniky Křesťanové, Tomáše Langáška, Jiřího Přibáně, Kateřiny Ronovské, Dity Řepkové, Jana Svatoně, Pavla Šámala, Jana Wintra a Daniely Zemanové o návrhu pléna na zrušení § 135 a § 2894 odst. 2 zákona č. 89/2012 Sb., občanský zákoník, za účasti Parlamentu České republiky, jehož jménem jednají Poslanecká sněmovna a Senát, jako účastníka řízení, takto:\r\n\n\n\r\nI. Návrh se zamítá.II. Účinná ochrana dobré pověsti právnických osob ústavně zaručená článkem 10 odst. 1 Listiny základních práv a svobod vyžaduje analogické 

In [ ]:
ECLI:CZ:US:2025:Pl.US.26.24.1
https://nalus.usoud.cz:443/Search/GetText.aspx?sz=Pl-26-24_1

ECLI:CZ:US:2023:Pl.US-st.59.23.1
https://nalus.usoud.cz:443/Search/GetText.aspx?sz=St-59-23_1

